In [ ]:
# Instalação de bibliotecas necessárias
pip install matplotlib numpy pandas scikit-learn

# Importando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.datasets import make_moons


In [ ]:
# Função para carregar os datasets
def load_banana():
    url = 'https://raw.githubusercontent.com/sauloafoliveira/cclw-mlm/master/local_datasets/datasets-7627-10826-banana.csv'
    data = pd.read_csv(url)
    X = data.iloc[:, :-1].values  # Features
    y = data.iloc[:, -1].values   # Labels
    return X, y

def load_ripley():
    url = 'https://raw.githubusercontent.com/sauloafoliveira/cclw-mlm/master/local_datasets/rip.csv'
    data = pd.read_csv(url)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

def load_two_moon():
    X, y = make_moons(n_samples=1000, noise=0.1)
    return X, y

# Carregando os datasets
X_banana, y_banana = load_banana()
X_ripley, y_ripley = load_ripley()
X_two_moon, y_two_moon = load_two_moon()

# Visualizando as primeiras linhas de cada dataset
print("Banana Dataset:", X_banana.shape, y_banana.shape)
print("Ripley Dataset:", X_ripley.shape, y_ripley.shape)
print("Two Moons Dataset:", X_two_moon.shape, y_two_moon.shape)


In [ ]:
# Funções de ativação e suas derivadas
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Definindo a arquitetura da MLP
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Inicializando pesos
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros((1, self.output_size))
    
    def forward(self, X):
        # Passagem para a camada oculta
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        
        # Passagem para a camada de saída
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        
        return self.a2
    
    def backward(self, X, y):
        # Cálculo do erro
        output_error = self.a2 - y
        output_delta = output_error * sigmoid_derivative(self.a2)
        
        # Cálculo do erro da camada oculta
        hidden_error = output_delta.dot(self.W2.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.a1)
        
        # Atualização dos pesos
        self.W2 -= self.a1.T.dot(output_delta) * self.learning_rate
        self.b2 -= np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate
        self.W1 -= X.T.dot(hidden_delta) * self.learning_rate
        self.b1 -= np.sum(hidden_delta, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y)
            
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - output))  # Mean Squared Error
                print(f'Epoch {epoch}, Loss: {loss:.4f}')
    
    def predict(self, X):
        return self.forward(X)


In [ ]:
# Divisão dos dados em treinamento e teste (70%/30%)
X_train_banana, X_test_banana, y_train_banana, y_test_banana = train_test_split(X_banana, y_banana, test_size=0.3)
X_train_ripley, X_test_ripley, y_train_ripley, y_test_ripley = train_test_split(X_ripley, y_ripley, test_size=0.3)
X_train_two_moon, X_test_two_moon, y_train_two_moon, y_test_two_moon = train_test_split(X_two_moon, y_two_moon, test_size=0.3)

# Inicializando e treinando o modelo
mlp_banana = MLP(input_size=X_train_banana.shape[1], hidden_size=10, output_size=1)
mlp_banana.train(X_train_banana, y_train_banana, epochs=1000)

mlp_ripley = MLP(input_size=X_train_ripley.shape[1], hidden_size=10, output_size=1)
mlp_ripley.train(X_train_ripley, y_train_ripley, epochs=1000)

mlp_two_moon = MLP(input_size=X_train_two_moon.shape[1], hidden_size=10, output_size=1)
mlp_two_moon.train(X_train_two_moon, y_train_two_moon, epochs=1000)


In [ ]:
# Função para calcular as métricas
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    predictions = np.round(predictions)  # Arredondando para 0 ou 1
    
    accuracy = accuracy_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions)
    
    return accuracy, recall, precision, f1, auc

# Avaliando os modelos
results_banana = evaluate_model(mlp_banana, X_test_banana, y_test_banana)
results_ripley = evaluate_model(mlp_ripley, X_test_ripley, y_test_ripley)
results_two_moon = evaluate_model(mlp_two_moon, X_test_two_moon, y_test_two_moon)

print("Banana Dataset:", results_banana)
print("Ripley Dataset:", results_ripley)
print("Two Moons Dataset:", results_two_moon)


In [ ]:
# Função para plotar a matriz de confusão
def plot_confusion_matrix(model, X_test, y_test, dataset_name):
    predictions = model.predict(X_test)
    predictions = np.round(predictions)
    cm = confusion_matrix(y_test, predictions)
    
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix - {dataset_name}')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, [0, 1])
    plt.yticks(tick_marks, [0, 1])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Visualizando as matrizes de confusão
plot_confusion_matrix(mlp_banana, X_test_banana, y_test_banana, "Banana")
plot_confusion_matrix(mlp_ripley, X_test_ripley, y_test_ripley, "Ripley")
plot_confusion_matrix(mlp_two_moon, X_test_two_moon, y_test_two_moon, "Two Moons")
